In [1864]:
import pandas as pd
import math
import numpy as np
from shapely.geometry import Point, Polygon
import copy
import time
import re
from datetime import date, datetime, timedelta
from shapely.geometry import Point, Polygon
import copy
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer

<h1>Do not amend this template.<h1>
<h1>Make a copy and adjust values in the below cell accordingly.<h1>
<h1>No other cell requires adjustment.<h1>

In [1865]:
neighbourhood_data_name = "SF_neighborhoods.csv"
sfpd_data_name = "incidents_new_data.csv" #current range 12/11/2020 to 28/01/2021
business_data_name = "business_new_data.csv"
final_data_name = "new_final_data.csv"
start_day = 12
start_month = 11
start_year = 2020
end_day = 28
end_month = 1
end_year = 2021

In [1866]:
start_date_string = str(start_year)+'-'+str(start_month)+'-'+str(start_day)
start_date = date(start_year,start_month,start_day)
one_year_before_start_date = start_date - timedelta(days=365)
end_date = date (end_year,end_month,end_day)
date_after_end_date = end_date + timedelta(days=1)
future_year_increment = 50
future_date = str(end_year+future_year_increment)+'-'+str(end_month)+'-'+str(end_day)
future_year = end_year + future_year_increment
business_start_date = str(start_month)+'-'+str(start_day)+'-'+str(start_year)
business_end_date = str(end_month)+'-'+str(end_day)+'-'+str(end_year)

<h1> Get neighbourhood data <h1>

In [1867]:
def open_neighborhoods():
    df = pd.read_csv(neighbourhood_data_name)
    return df

In [1868]:
def get_neighborhoods(df):
    neighborhoods = []
    num_neighborhoods = len(df.index)
    for x in range(0,num_neighborhoods):
        neighborhoods.append(df.iloc[x,2])
    return neighborhoods

In [1869]:
def generate_neighborhood_polygons(df):
    neighborhoods = []
    num_neighborhoods = len(df.index)
    for x in range(0,num_neighborhoods):
        raw_polygon_data = df.iloc[x,1]
        neighborhood_polygon = generate_polygon(raw_polygon_data)
        neighborhood_name = df.iloc[x,2]
        neighborhood_tuple = (neighborhood_polygon,neighborhood_name)
        neighborhoods.append(neighborhood_tuple)
    return neighborhoods

In [1870]:
def generate_polygon(raw_polygon_data):
    raw_polygon_data = raw_polygon_data[16:-3]
    coords = ''
    coords_list = []
    for char in raw_polygon_data:
        if char == ' ' or char == ',':
            if coords != '':
                coords_list.append(float(coords))
            coords = ''
        else:
            coords = coords+char
    coords_list.append(float(coords))
    x = 0
    tuple_list = []
    while x < len(coords_list):
        next_tuple = (coords_list[x+1],coords_list[x])
        tuple_list.append(next_tuple)
        x+=2
    polygon = Polygon(tuple_list)
    return polygon

In [1871]:
df = open_neighborhoods()
neighborhoods = get_neighborhoods(df)
num_neighbourhoods = len(neighborhoods)
neighborhood_polygons = generate_neighborhood_polygons(df)

<h1> Get incident report data <h1>

In [1872]:
def open_sfpd_data():
    df = pd.read_csv(sfpd_data_name)
    return df

In [1873]:
def remove_uncategorised_incidents(df):
    #identify total number of records and remove uncategorised crimes
    total_records = len(df.index)
    categorised_crimes = df['Incident Category'].count()
    uncategorised_crimes = total_records - categorised_crimes
    if uncategorised_crimes != 0:
        #drop blank incident categories and recheck values
        df = df.dropna(subset=['Incident Category'])
        df = remove_uncategorised_incidents(df)
    return df

In [1874]:
def remove_rows_with_missing_values(df):
    df = df.dropna()
    return df

In [1875]:
def drop_unnecessary_incident_columns(df):
    df = df.drop(columns=['Report Datetime','Row ID', 'Incident ID',
                          'Incident Number','CAD Number','Report Type Code',
                          'Report Type Description', 'Filed Online',
                          'Incident Code','Incident Subcategory',
                          'Incident Description', 'Resolution', 'Intersection',
                          'CNN','Police District','Analysis Neighborhood',
                          'Supervisor District', 'point', 'SF Find Neighborhoods',
                          'Current Police Districts', 'Current Supervisor Districts',
                          'Analysis Neighborhoods','HSOC Zones as of 2018-06-05',
                          'OWED Public Spaces',
                          'Central Market/Tenderloin Boundary Polygon - Updated',
                          'Parks Alliance CPSI (27+TL sites)','ESNCAG - Boundary File',
                          'Areas of Vulnerability, 2016'])
    return df

In [1876]:
def remove_rows_of_non_crime(df):
    df = df.loc[~df['Incident Category'].str.contains('Case Closure')]
    df = df.loc[~df['Incident Category'].str.contains('Courtesy Report')]
    df = df.loc[~df['Incident Category'].str.contains('Lost Property')]
    df = df.loc[~df['Incident Category'].str.contains('Missing Person')]
    df = df.loc[~df['Incident Category'].str.contains('Non-Criminal')]
    df = df.loc[~df['Incident Category'].str.contains('Suicide')]
    df = df.loc[~df['Incident Category'].str.contains('Vehicle Misplaced')]
    df = df.loc[~df['Incident Category'].str.contains('Warrant')]
    return df

In [1877]:
def remove_crime_prior_to_start_date(df):
    df['Incident Date'] = pd.to_datetime(df['Incident Date'],dayfirst=True)
    df = df[~(df['Incident Date'].dt.date < one_year_before_start_date)]
    return df

In [1878]:
def convert_gps_to_neighborhood(df,neighborhood_polygons):
    df['Neighborhood'] = np.nan
    num_records = len(df.index)
    for incident_num in range(0,num_records):
        df.iloc[incident_num,8] = ''
        shortest_distance = 999999999
        closest_neighborhood = ''
        latitude = float(df.iloc[incident_num,6])
        longitude = float(df.iloc[incident_num,7])
        point = Point(latitude,longitude)
        for poly_tuple in neighborhood_polygons:
            if poly_tuple[0].contains(point) or poly_tuple[0].touches(point):
                df.iloc[incident_num,8] = poly_tuple[1]
                break
            else:
                distance_to_neighborhood = point.distance(poly_tuple[0])
                if distance_to_neighborhood < shortest_distance:
                    shortest_distance = distance_to_neighborhood
                    closest_neighborhood = poly_tuple[1]
        if df.iloc[incident_num,8] == '':
            #place incident in closest neighborhood
            df.iloc[incident_num,8] = closest_neighborhood
            #shortest distance is over 1 mile away
            if shortest_distance > 0.018:
                print('WARNING!!! GPS coordinates for row',incident_num,'do not appear to be within 1 mile of a San Francisco Neighbourhood')
    return df

In [1879]:
def remove_gps_coordinates(df):
    df = df.drop(columns=['Latitude','Longitude'])
    return df

In [1880]:
def get_reports_per_day_per_neighborhood(df):
    df.to_csv("sfpd_pre_per_day.csv", index = False)
    df['Todays Reports'] = 1
    df['Incident Date'] = pd.to_datetime(df['Incident Date'],dayfirst=True)
    df = df.groupby(['Incident Date',
                     'Incident Year', 
                     'Neighborhood',
                     'Incident Day of Week'
                    ]).count()
    df = df.drop(columns=[ 'Incident Datetime','Incident Time',
                           'Incident Category'])
    df = df.reset_index()
    df.to_csv("sfpd_post_per_day.csv", index = False)
    return df

In [1881]:
def check_for_missing_neighborhoods(df,neighborhoods):
    new_rows = pd.DataFrame()
    num_records = len(df.index)
    current_date = df.iloc[0,0]
    zero_crime_neighborhoods = copy.deepcopy(neighborhoods)
    for record in range(0,num_records):
        zero_crime_neighborhoods.remove(df.iloc[record,2])
        if record != num_records-1:
            if df.iloc[record+1,0] != current_date:
                for neighborhood in zero_crime_neighborhoods:
                    new_row = {'Incident Date' : current_date,
                               'Incident Year' : df.iloc[record-1,1],
                               'Neighborhood' : neighborhood,
                               'Incident Day of Week' : df.iloc[record-1,3],
                               'Todays Reports' : 0}
                    new_rows = new_rows.append(new_row,ignore_index=True)
                zero_crime_neighborhoods = copy.deepcopy(neighborhoods)
                current_date = df.iloc[record+1,0]
    for neighborhood in zero_crime_neighborhoods:
        new_row = {'Incident Date' : current_date,
                    'Incident Year' : df.iloc[len(df.index)-1,1],
                    'Neighborhood' : neighborhood,
                    'Incident Day of Week' : df.iloc[len(df.index)-1,3],
                    'Todays Reports' : 0}
        new_rows = new_rows.append(new_row,ignore_index=True)
    frames = [df,new_rows]
    new_record = pd.concat(frames)
    new_record = new_record.sort_values(['Incident Date','Neighborhood'])
    return new_record

In [1882]:
def get_last_14_days(df):
    transitional_data = df.shift(periods=(num_neighbourhoods*7))
    transitional_data = transitional_data['Last 7 days reports']
    return transitional_data

In [1883]:
def get_last_28_days(df):
    transitional_data = df.shift(periods=(num_neighbourhoods*14))
    transitional_data = transitional_data['Last 14 days reports']
    return transitional_data

In [1884]:
def generate_new_date_data(df):
    df['Reports 1 day ago'] = shift_crimes(df,1)
    df['Reports 2 days ago'] = shift_crimes(df,2)
    df['Reports 3 days ago'] = shift_crimes(df,3)
    df['Reports 4 days ago'] = shift_crimes(df,4)
    df['Reports 5 days ago'] = shift_crimes(df,5)
    df['Reports 6 days ago'] = shift_crimes(df,6)
    df['Reports 7 days ago'] = shift_crimes(df,7)
    df['Reports 14 days ago'] = shift_crimes(df,14)
    df['Reports 30 days ago'] = shift_crimes(df,30)
    df['Reports 365 days ago'] = shift_crimes(df,365)
    df['Last 7 days reports'] = (df['Reports 1 day ago'] + df['Reports 2 days ago'] + df['Reports 3 days ago']
                         + df['Reports 4 days ago'] + df['Reports 5 days ago']
                         + df['Reports 6 days ago'] + df['Reports 7 days ago'])
    df['Last 14 days reports'] = (df['Last 7 days reports'] + get_last_14_days(df))
    df['Last 28 days reports'] = (df['Last 14 days reports'] + get_last_28_days(df))
    df = df.dropna()
    return df

In [1885]:
def reset_indexes(df):
    #print("Resetting indexes")
    #print(df)
    df.reset_index(drop=True, inplace=True)
    #print("Returning reset df")
    #print(df)
    return df

In [1886]:
def shift_crimes(df,num_days):
    #print("Incoming df to shift by days: " + str(num_days))
    #print(df)
    transitional_data = df.shift(periods=(num_neighbourhoods*num_days))
    #print("Transitional Data:")
    #print(transitional_data)
    transitional_data = transitional_data['Todays Reports']
    #print("Transitional Data:")
    #print(transitional_data)
    return transitional_data

In [1887]:
# Incident data generation
sfpd_df = open_sfpd_data()
sfpd_df = remove_uncategorised_incidents(sfpd_df)
sfpd_df = drop_unnecessary_incident_columns(sfpd_df)
sfpd_df = remove_rows_with_missing_values(sfpd_df)
sfpd_df = remove_rows_of_non_crime(sfpd_df)
#sfpd_df.to_csv("mid_process5.csv", index = False)

In [1888]:
#sfpd_df = pd.read_csv("mid_process5.csv")
sfpd_df = remove_crime_prior_to_start_date(sfpd_df)
#sfpd_df.to_csv("mid_process6.csv", index = False)
sfpd_df = convert_gps_to_neighborhood(sfpd_df,neighborhood_polygons)
#sfpd_df.to_csv("mid_process7.csv", index = False)
sfpd_df = remove_gps_coordinates(sfpd_df)
#sfpd_df.to_csv("mid_process8.csv", index = False)

In [1889]:
#sfpd_df = pd.read_csv("mid_process.csv")
sfpd_df = get_reports_per_day_per_neighborhood(sfpd_df)
sfpd_df = check_for_missing_neighborhoods(sfpd_df,neighborhoods)
sfpd_df = reset_indexes(sfpd_df)
sfpd_df = generate_new_date_data(sfpd_df)
sfpd_df = reset_indexes(sfpd_df)

<h1>Get business data<h1>

In [1917]:
def open_businesses():
    df = pd.read_csv(business_data_name)
    return df

In [1918]:
def drop_unnecessary_columns(df):
    df = df.drop(columns=['Analysis Neighborhoods','Current Supervisor Districts',
                          'Current Police Districts','SF Find Neighborhoods','Neighborhoods',
                          'LIC Code Description','LIC Code','Transient Occupancy Tax','Parking Tax',
                          'NAICS Code','Mail State','Mail City','Location Id','Business Account Number',
                          'Business Start Date','Business End Date','Ownership Name','Street Address',
                          'City','State','Source Zipcode','Mail Address','Mail Zipcode',
                          'Supervisor District','Business Corridor','NAICS Code Description','UniqueID'])
    return df

In [1919]:
#adjusted crime dataset starts 01/07/2019, business closed one year prior to this are not required 
def drop_end_dates_before_uber_legislation_change(df):
    # give none closed businesses an arbitray future closure date
    df['Location End Date'] = df['Location End Date'].fillna(future_date)
    #convert all date strings to datetime
    df['Location End Date'] = pd.to_datetime(df['Location End Date'])
    df = df[~(df['Location End Date'].dt.date < one_year_before_start_date)]
    return df

In [1920]:
def determine_neighborhood_by_geolocation(df, neighborhood_polygons,neighborhoods):
    #replace blank gps coords with arbitrary point a significant distance from san francisco, in the ocean
    df['Business Location'] = df['Business Location'].fillna('POINT (-120.0 30.00)')
    df['Business Location'] = df['Business Location'].str[7:-1]
    df[['Longitude','Latitude']] = df['Business Location'].str.split(" ", expand = True)
    df = convert_gps_to_neighborhood(df,neighborhood_polygons,neighborhoods)
    return df

In [1921]:
def convert_gps_to_neighborhood(df,neighborhood_polygons,neighborhoods):
    #print(df)
    #df.to_csv("saved_data_to_check", index = False)
    df['Neighborhood'] = np.nan
    num_records = len(df.index)
    for business_num in range(0,num_records):
        df.iloc[business_num,7] = ''
        if df.iloc[business_num,4] == '-120.0 30.00':
            for neighborhood in neighborhoods:
                if df.iloc[business_num,3] == neighborhood:
                    df.iloc[business_num,7] = neighborhood
            if df.iloc[business_num,7] == '':
                df.iloc[business_num,7] = 'no gps'     
        else:
            shortest_distance = 999999999
            closest_neighborhood = ''
            latitude = float(df.iloc[business_num,6])
            longitude = float(df.iloc[business_num,5])
            point = Point(latitude,longitude)
            for poly_tuple in neighborhood_polygons:
                if poly_tuple[0].contains(point) or poly_tuple[0].touches(point):
                    df.iloc[business_num,7] = poly_tuple[1]
                    break
                else:
                    distance_to_neighborhood = point.distance(poly_tuple[0])
                    if distance_to_neighborhood < shortest_distance:
                        shortest_distance = distance_to_neighborhood
                        closest_neighborhood = poly_tuple[1]
            if df.iloc[business_num,7] == '':
                #if less than half a mile from a neighborhood polygon and sufficiently north to be san francisco
                if shortest_distance < 0.018 and latitude > 37.709126:
                    df.iloc[business_num,7] = closest_neighborhood
    df.reset_index(drop=True, inplace=True)
    df = df[df.Neighborhood != '']
    df = df[df.Neighborhood != 'no gps']
    return df

In [1922]:
def cleanup_columns(df):
    #print("cleanup")
    #print(df.head())
    df = df.drop(columns=['Neighborhoods - Analysis Boundaries','Business Location',
                          'Longitude','Latitude'])
    #print(df.head())
    return df

In [1923]:
def get_closures_per_day_per_neighborhood(closures):
    #print("closures per day")
    #print(closures.head())
    closures['Closures'] = 1
    closures['Location End Date'] = pd.to_datetime(closures['Location End Date'])
    closures = closures[closures['Location End Date'].dt.year < future_year]
    closures = closures.groupby(['Location End Date','Neighborhood']).count()
    closures = closures.drop(columns=[ 'DBA Name','Location Start Date'])
    closures = closures.reset_index()
    #print(closures.head())
    return closures

In [1924]:
def add_zero_closure_neighborhoods(closures, neighborhoods):
    new_rows = pd.DataFrame()
    num_records = len(closures.index)
    current_date = closures.iloc[0,0]
    zero_closure_neighborhoods = copy.deepcopy(neighborhoods)
    for record in range(0,num_records):
        while closures.iloc[record,0] != current_date:
            for neighborhood in zero_closure_neighborhoods:
                new_row = {'Location End Date' : current_date,
                                   'Neighborhood' : neighborhood,
                                   'Closures' : 0}
                new_rows = new_rows.append(new_row,ignore_index=True)
            current_date = current_date + timedelta(days=1)
        if closures.iloc[record,0] == current_date:
            zero_closure_neighborhoods.remove(closures.iloc[record,1])
            if record != num_records-1:
                if closures.iloc[record+1,0] != current_date:
                    for neighborhood in zero_closure_neighborhoods:
                        new_row = {'Location End Date' : current_date,
                                   'Neighborhood' : neighborhood,
                                   'Closures' : 0}
                        new_rows = new_rows.append(new_row,ignore_index=True)
                    zero_closure_neighborhoods = copy.deepcopy(neighborhoods)
                    current_date = current_date + timedelta(days=1)
        if current_date == date_after_end_date:
            break
    for neighborhood in zero_closure_neighborhoods:
        new_row = {'Location End Date' : current_date,
                    'Neighborhood' : neighborhood,
                    'Closures' : 0}
        new_rows = new_rows.append(new_row,ignore_index=True)
    frames = [closures,new_rows]
    new_record = pd.concat(frames)
    new_record = new_record.sort_values(['Location End Date','Neighborhood'])
    return new_record

In [1925]:
def shift_closures(closures,num_days):
    transitional_data = closures.shift(periods=(num_neighbourhoods*num_days))
    transitional_data = transitional_data['Closures']
    return transitional_data

In [1926]:
def get_last_14_days_closures(closures):
    transitional_data = closures.shift(periods=(num_neighbourhoods*7))
    transitional_data = transitional_data['Last 7 days closures']
    return transitional_data

In [1927]:
def get_last_28_days_closures(closures):
    transitional_data = closures.shift(periods=(num_neighbourhoods*14))
    transitional_data = transitional_data['Last 14 days closures']
    return transitional_data

In [1928]:
def generate_new_closure_date_data(closures):
    closures['Closures 1 day ago'] = shift_closures(closures,1)
    closures['Closures 2 days ago'] = shift_closures(closures,2)
    closures['Closures 3 days ago'] = shift_closures(closures,3)
    closures['Closures 4 days ago'] = shift_closures(closures,4)
    closures['Closures 5 days ago'] = shift_closures(closures,5)
    closures['Closures 6 days ago'] = shift_closures(closures,6)
    closures['Closures 7 days ago'] = shift_closures(closures,7)
    closures['Closures 14 days ago'] = shift_closures(closures,14)
    closures['Closures 30 days ago'] = shift_closures(closures,30)
    closures['Closures 365 days ago'] = shift_closures(closures,365)
    closures['Last 7 days closures'] = (closures['Closures 1 day ago']
                                        + closures['Closures 2 days ago']
                                        + closures['Closures 3 days ago']
                                        + closures['Closures 4 days ago']
                                        + closures['Closures 5 days ago']
                                        + closures['Closures 6 days ago']
                                        + closures['Closures 7 days ago'])
    closures['Last 14 days closures'] = (closures['Last 7 days closures']
                                         + get_last_14_days_closures(closures))
    closures['Last 28 days closures'] = (closures['Last 14 days closures']
                                         + get_last_28_days_closures(closures))
    closures = closures.dropna()
    closures = closures[closures['Location End Date'] >= business_start_date]
    closures = closures[closures['Location End Date'] <= business_end_date]
    return closures

In [1929]:
def get_openings_per_day_per_neighborhood(openings):
    openings['Openings'] = 1
    openings['Location Start Date'] = pd.to_datetime(openings['Location Start Date'])
    openings = openings[openings['Location Start Date'].dt.year >= start_year-1]
    openings = openings.groupby(['Location Start Date','Neighborhood']).count()
    openings = openings.drop(columns=[ 'DBA Name','Location End Date'])
    openings = openings.reset_index()
    return openings

In [1930]:
def add_zero_opening_neighborhoods(openings, neighborhoods):
    new_rows = pd.DataFrame()
    num_records = len(openings.index)
    current_date = openings.iloc[0,0]
    zero_opening_neighborhoods = copy.deepcopy(neighborhoods)
    for record in range(0,num_records):
        while openings.iloc[record,0] != current_date:
            for neighborhood in zero_opening_neighborhoods:
                new_row = {'Location Start Date' : current_date,
                                   'Neighborhood' : neighborhood,
                                   'Openings' : 0}
                new_rows = new_rows.append(new_row,ignore_index=True)
            current_date = current_date + timedelta(days=1)
        if openings.iloc[record,0] == current_date:
            zero_opening_neighborhoods.remove(openings.iloc[record,1])
            if record != num_records-1:
                if openings.iloc[record+1,0] != current_date:
                    for neighborhood in zero_opening_neighborhoods:
                        new_row = {'Location Start Date' : current_date,
                                   'Neighborhood' : neighborhood,
                                   'Openings' : 0}
                        new_rows = new_rows.append(new_row,ignore_index=True)
                    zero_opening_neighborhoods = copy.deepcopy(neighborhoods)
                    current_date = current_date + timedelta(days=1)
        if current_date == date_after_end_date:
            break
    for neighborhood in zero_opening_neighborhoods:
        new_row = {'Location Start Date' : current_date,
                    'Neighborhood' : neighborhood,
                    'Openings' : 0}
        new_rows = new_rows.append(new_row,ignore_index=True)
    frames = [openings,new_rows]
    new_record = pd.concat(frames)
    new_record = new_record.sort_values(['Location Start Date','Neighborhood'])
    return new_record

In [1931]:
def shift_openings(openings,num_days):
    transitional_data = openings.shift(periods=(num_neighbourhoods*num_days))
    transitional_data = transitional_data['Openings']
    return transitional_data

In [1932]:
def get_last_14_days_openings(openings):
    transitional_data = openings.shift(periods=(num_neighbourhoods*7))
    transitional_data = transitional_data['Last 7 days openings']
    return transitional_data

In [1933]:
def get_last_28_days_openings(openings):
    transitional_data = openings.shift(periods=(num_neighbourhoods*14))
    transitional_data = transitional_data['Last 14 days openings']
    return transitional_data

In [1934]:
def generate_new_opening_date_data(openings):
    openings['Openings 1 day ago'] = shift_openings(openings,1)
    openings['Openings 2 days ago'] = shift_openings(openings,2)
    openings['Openings 3 days ago'] = shift_openings(openings,3)
    openings['Openings 4 days ago'] = shift_openings(openings,4)
    openings['Openings 5 days ago'] = shift_openings(openings,5)
    openings['Openings 6 days ago'] = shift_openings(openings,6)
    openings['Openings 7 days ago'] = shift_openings(openings,7)
    openings['Openings 14 days ago'] = shift_openings(openings,14)
    openings['Openings 30 days ago'] = shift_openings(openings,30)
    openings['Openings 365 days ago'] = shift_openings(openings,365)
    openings['Last 7 days openings'] = (openings['Openings 1 day ago']
                                        + openings['Openings 2 days ago']
                                        + openings['Openings 3 days ago']
                                        + openings['Openings 4 days ago']
                                        + openings['Openings 5 days ago']
                                        + openings['Openings 6 days ago']
                                        + openings['Openings 7 days ago'])
    openings['Last 14 days openings'] = (openings['Last 7 days openings']
                                         + get_last_14_days_openings(openings))
    openings['Last 28 days openings'] = (openings['Last 14 days openings']
                                         + get_last_28_days_openings(openings))
    openings = openings.dropna()
    openings = openings[openings['Location Start Date'] >= business_start_date]
    openings = openings[openings['Location Start Date'] <= business_end_date]
    return openings

In [1935]:
def generate_dates_for_each_business(df):
    df['Date'] = np.nan
    df['Date']= pd.to_datetime(df['Date'])
    df['Location Start Date']= pd.to_datetime(df['Location Start Date'])
    df['Location End Date']= pd.to_datetime(df['Location End Date'])
    new_df = pd.DataFrame()
    final_df = pd.DataFrame()
    num_records = len(df.index)
    current_date = one_year_before_start_date #one year prior to beginning of SFPD data
    #end_date = end_date #final date of SFPD data
    delta = timedelta(days = 1)
    while current_date <= end_date:
        #print (current_date.strftime("%Y-%m-%d"))
        num_records = len(df.index)
        
        #new_df = (df.loc[(df['Location Start Date'] <= current_date) & (current_date < df['Location End Date'])])
        
        new_df = (df.loc[(pd.to_datetime(df['Location Start Date']).dt.date <= current_date) 
                         & (current_date < pd.to_datetime(df['Location End Date']).dt.date)])
        
        
        
        new_df['Date'] = current_date
        final_df = pd.concat((new_df,final_df))
        current_date += delta
    final_df['Number of businesses'] = 1
    final_df = final_df.groupby(['Date','Neighborhood']).count()
    final_df = final_df.drop(columns=[ 'DBA Name','Location Start Date',
                           'Location End Date'])
    return final_df

In [1936]:
def shift_days(df,num_days):
    transitional_data = df.shift(periods=(num_neighbourhoods*num_days))
    transitional_data = transitional_data['Number of businesses']
    return transitional_data

In [1937]:
def get_last_14_days(df):
    transitional_data = df.shift(periods=(num_neighbourhoods*7))
    transitional_data = transitional_data['Last 7 days']
    return transitional_data

In [1938]:
def get_last_28_days(df):
    transitional_data = df.shift(periods=(num_neighbourhoods*14))
    transitional_data = transitional_data['Last 14 days']
    return transitional_data

In [1939]:
def generate_new_date_data(df):
    df['Businesses 1 day ago'] = shift_days(df,1)
    df['Businesses 2 days ago'] = shift_days(df,2)
    df['Businesses 3 days ago'] = shift_days(df,3)
    df['Businesses 4 days ago'] = shift_days(df,4)
    df['Businesses 5 days ago'] = shift_days(df,5)
    df['Businesses 6 days ago'] = shift_days(df,6)
    df['Businesses 7 days ago'] = shift_days(df,7)
    df['Businesses 14 days ago'] = shift_days(df,14)
    df['Businesses 30 days ago'] = shift_days(df,30)
    df['Businesses 365 days ago'] = shift_days(df,365)
    df = df.dropna()
    return df

In [ ]:
business_df = open_businesses()
business_df = drop_unnecessary_columns(business_df)
business_df.to_csv("bus_progress1.csv", index = False)
business_df = drop_end_dates_before_uber_legislation_change(business_df)
business_df.to_csv("bus_progress2.csv", index = False)
business_df = determine_neighborhood_by_geolocation(business_df, neighborhood_polygons,neighborhoods)
business_df.to_csv("bus_progress3.csv", index = False)
business_df = cleanup_columns(business_df)
business_df.to_csv("bus_progress4.csv", index = False)
closures = business_df.copy(deep=True)
closures.to_csv("clo_progress1.csv", index = False)
closures = get_closures_per_day_per_neighborhood(closures)
closures.to_csv("clo_progress2.csv", index = False)
closures = add_zero_closure_neighborhoods(closures,neighborhoods)
closures.to_csv("clo_progress3.csv", index = False)
closures = generate_new_closure_date_data(closures)
closures.to_csv("clo_progress4.csv", index = False)
openings = business_df.copy(deep=True)
openings.to_csv("ope_progress1.csv", index = False)
openings = get_openings_per_day_per_neighborhood(openings)
openings.to_csv("ope_progress2.csv", index = False)
openings = add_zero_opening_neighborhoods(openings,neighborhoods)
openings.to_csv("ope_progress3.csv", index = False)
openings = generate_new_opening_date_data(openings)
openings.to_csv("ope_progress4.csv", index = False)
business_df = generate_dates_for_each_business(business_df)
business_df.to_csv("bus_progress5.csv", index = False)
business_df = generate_new_date_data(business_df)
business_df.to_csv("bus_progress6.csv", index = False)

c:\users\chris\appdata\local\programs\python\python39\lib\site-packages\IPython\core\interactiveshell.py:3338: DtypeWarning: Columns (12,13,14,15) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


<h1>Merge dataframes<h1>

In [ ]:
def finalise_data(sfpd_data,businesses,openings,closures):
    sfpd_data.rename(columns={"1 day ago": "Crime 1 day ago",
                              "2 days ago": "Crime 2 days ago",
                              "3 days ago": "Crime 3 days ago",
                              "4 days ago": "Crime 4 days ago",
                              "5 days ago": "Crime 5 days ago",
                              "6 days ago": "Crime 6 days ago",
                              "7 days ago": "Crime 7 days ago",
                              "14 days ago": "Crime 14 days ago",
                              "30 days ago": "Crime 30 days ago",
                              "365 days ago": "Crime 365 days ago",
                              "Last 7 days": "Last 7 days crime",
                              "Last 14 days": "Last 14 days crime",
                              "Last 28 days": "Last 28 days crime"})
    sfpd_data['Number of businesses'] = businesses['Number of businesses']
    sfpd_data['Businesses 1 day ago'] = businesses['Businesses 1 day ago']
    sfpd_data['Businesses 2 days ago'] = businesses['Businesses 2 days ago']
    sfpd_data['Businesses 3 days ago'] = businesses['Businesses 3 days ago']
    sfpd_data['Businesses 4 days ago'] = businesses['Businesses 4 days ago']
    sfpd_data['Businesses 5 days ago'] = businesses['Businesses 5 days ago']
    sfpd_data['Businesses 6 days ago'] = businesses['Businesses 6 days ago']
    sfpd_data['Businesses 7 days ago'] = businesses['Businesses 7 days ago']
    sfpd_data['Businesses 14 days ago'] = businesses['Businesses 14 days ago']
    sfpd_data['Businesses 30 days ago'] = businesses['Businesses 30 days ago']
    sfpd_data['Businesses 365 days ago'] = businesses['Businesses 365 days ago']
    sfpd_data['Number of closures'] = closures['Closures']
    sfpd_data['Closures 1 day ago'] = closures['Closures 1 day ago']
    sfpd_data['Closures 2 days ago'] = closures['Closures 2 days ago']
    sfpd_data['Closures 3 days ago'] = closures['Closures 3 days ago']
    sfpd_data['Closures 4 days ago'] = closures['Closures 4 days ago']
    sfpd_data['Closures 5 days ago'] = closures['Closures 5 days ago']
    sfpd_data['Closures 6 days ago'] = closures['Closures 6 days ago']
    sfpd_data['Closures 7 days ago'] = closures['Closures 7 days ago']
    sfpd_data['Closures 14 days ago'] = closures['Closures 14 days ago']
    sfpd_data['Closures 30 days ago'] = closures['Closures 30 days ago']
    sfpd_data['Closures 365 days ago'] = closures['Closures 365 days ago']
    sfpd_data['Last 7 days closures'] = closures['Last 7 days closures']
    sfpd_data['Last 14 days closures'] = closures['Last 14 days closures']
    sfpd_data['Last 28 days closures'] = closures['Last 28 days closures']
    sfpd_data['Number of openings'] = openings['Openings']
    sfpd_data['Openings 1 day ago'] = openings['Openings 1 day ago']
    sfpd_data['Openings 2 days ago'] = openings['Openings 2 days ago']
    sfpd_data['Openings 3 days ago'] = openings['Openings 3 days ago']
    sfpd_data['Openings 4 days ago'] = openings['Openings 4 days ago']
    sfpd_data['Openings 5 days ago'] = openings['Openings 5 days ago']
    sfpd_data['Openings 6 days ago'] = openings['Openings 6 days ago']
    sfpd_data['Openings 7 days ago'] = openings['Openings 7 days ago']
    sfpd_data['Openings 14 days ago'] = openings['Openings 14 days ago']
    sfpd_data['Openings 30 days ago'] = openings['Openings 30 days ago']
    sfpd_data['Openings 365 days ago'] = openings['Openings 365 days ago']
    sfpd_data['Last 7 days openings'] = openings['Last 7 days openings']
    sfpd_data['Last 14 days openings'] = openings['Last 14 days openings']
    sfpd_data['Last 28 days openings'] = openings['Last 28 days openings']
    sfpd_data = sfpd_data.drop(columns=['Incident Date','Incident Year'])
    return sfpd_data

In [ ]:
def convert_day_of_week(df):
    dummies = pd.get_dummies(df['Incident Day of Week'])
    merged = pd.concat([df,dummies],axis='columns')
    final = merged.drop(['Incident Day of Week','Monday'],axis='columns')
    return final

In [ ]:
def convert_neighborhood(df):
    dummies = pd.get_dummies(df['Neighborhood'])
    merged = pd.concat([df,dummies],axis='columns')
    final = merged.drop(['Neighborhood','Ashbury Heights'],axis='columns')
    return final

In [ ]:
def normalise_data(df):
    y_data = df[['Todays Reports']]
    x_data = df.drop(columns=['Todays Reports'])
    x_data = x_data.apply(lambda x: (x - x.min(axis = 0)) / (x.max(axis = 0) - x.min(axis = 0)))
    x_data['Todays Reports'] = y_data['Todays Reports']
    return x_data

In [ ]:
print("Incidents to merge")
print(sfpd_df)
print("Business to merge")
print(business_df)
print("Openings to merge")
print(openings)
print("Closures to merge")
print(closures)


In [ ]:
sfpd_df.shape

In [ ]:
business_df.shape

In [ ]:
final_data = finalise_data(sfpd_df,business_df,openings,closures)
final_data = convert_neighborhood(final_data)
final_data = convert_day_of_week(final_data)
final_data = normalise_data(final_data)
final_data
final_data.to_csv(final_data_name, index = False)

In [1914]:
sfpd_df.to_csv(final_data_name, index = False)